<a href="https://colab.research.google.com/github/hedayaahmed/ITNPAI1_Vehicles_Detection/blob/main/2023_Spring_Assignment_AI1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

#University of Stirling - Spring 2023

## ITNPAI1 - Deep Learning for Vision and NLP (2022/3)

---


# 1. **Problem definition** 

This project presents deep learning based solution for vehicles detection including cars, motorcycles, buses, and trucks from wild images at street-level. The dataset is collected from two cities which are Cairo and Stirling.

[top](#scrollTo=4i5afvUbhmGo)
 

---
# 2. **Dataset creation**
202 images from Cairo and 200 images from Stirling are collected using **Mapillary API**. The first step was collecting sequence keys for each city and save them in seperate text file. Then, this [notebook](Hidden_landing_URL) was used to read these keys and send a request to **Mapillary API** to get response and download these images.

Cairo dataset includes 809 cars, 49 motorcycles, 15 buses, and 57 trucks, while Stirling dataset contains 548 cars, 2 motorcycles, 7 buses, and 19 trucks.

The dataset was annotated using Computer Vision Annotation Tool "CVAT". Then, it is exported in Yolo format and converted to PASCAL VOC format using this [notebook](Hidden_landing_URL) because the most recent version of CVAT is more accurate in bounding boxes extraction in Yolo and CVAT formats than in PASCAL VOC. The annotation consists of bounding boxes "xmin, ymin, xmax, ymax", object class, and filter the repeated, low quality, or empty images.

You must collect a minimum of **200 positive samples** from the study objects for each city (A and B). 
Note that, depending on the task being solved, it will also be necessary to: 

   (i) collect more samples - negative ones, for instance;

   (ii) annotating each image, delineating objects or creating bounding boxes. Planning and executing this correctly is important to ensure effective training of deep learning-based models.

Your dataset can be assembled from one or more of the following ways:

  - *M1* - Pictures taken by yourself on site (street view from cities A and B), with attention to anonymization issues (if it is the case). It is not allowed to assemble datasets containing people. Other sensitive patterns, such as license plates, must be properly hidden.

  - *M2* - Aerial satellite/drone images obtained from GIS and remote sensing platforms or public repositories. Be careful with unusual file formats that may be challenging to manipulate using basic image processing libraries. We recommend keeping or converting the images to jpg or png.

  - *M3* - Pictures taken from other public available datasets. Remember you are not allowed to use datasets containing people or other sensitive patterns/objects.

  - *M4* - Images crawled from the internet as a whole (social networks, webpages, etc), with special attention to use and copyrights.

  - *M5* - Textual and metadata you may need in your project, with special attention to use and copyrights (as always!).

**Important:** If you collect the images on your own or from aerial imagery repositories, it will be necessary to keep the geographic coordinates. If you collect from specific websites, please retain the source links. This information should be placed in a .csv file and made available along with the final dataset.

[top](#scrollTo=4i5afvUbhmGo)

---

# 4. **Dataloader**

Here you are required to implement all the code related to pre-processing, cleaning, de-noising and preparing the input images and metadata according to the necessary data structures as input to your pattern recognition module. We recommend using [PyTorch](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) or [Tensorflow (with Keras)](https://keras.io/getting_started/intro_to_keras_for_engineers/) as a base, but you are free to use any library or platform as long as it is well justified in the [final report](#scrollTo=ws14iV4Dp_vf).

[top](scrollTo=4i5afvUbhmGo)

In [ ]:
# Write your dataloader code here. Create more code cells if you find it necessary
!unzip /content/drive/MyDrive/Data_2.zip

In [ ]:
# defining the files directory and testing directory
files_dir = '/content/Data_2/Egypt'
test_dir = '/content/Data_2/Stirling'

In [ ]:
class VehiclesDetectionDataset(torch.utils.data.Dataset):
    
    def __init__(self, files_dir, width, height, transforms=None):
        self.files_dir = files_dir
        self.width = width
        self.height = height
        self.transforms = transforms  # If transformation is required, when transforms is not None
        
        self.classes_ = [_, 'car', 'motorcycle', 'bus', 'truck']  # Defining classes, a blank class is given for the background
        
        self.images = [img for img in sorted(os.listdir(files_dir)) if img[-4:]=='.jpg']
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.files_dir, img_name)
        
        # Reading the image
        img = cv2.imread(img_path)
        
        # Defining width and height
        wt = img.shape[1]
        ht = img.shape[0]
        
        # Converting image to RGB channel and normalizing the image
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img = cv2.resize(img, (self.width, self.height), cv2.INTER_AREA)
        img /= 255.0
        
        annot_name = img_name[:-4] + '.xml'
        annot_path = os.path.join(self.files_dir, annot_name)
        
        # Boxes to store the coordinate points of the bboxes
        boxes, labels = [], []
        
        tree = et.parse(annot_path)
        root = tree.getroot()
        
        # Box coordinates are extracted from the XML files for the given image size
        for member in root.findall('object'):
            labels.append(self.classes_.index(member.find('name').text))
            
            xmin = float(member.find('bndbox').find('xmin').text)
            xmax = float(member.find('bndbox').find('xmax').text)
            ymin = float(member.find('bndbox').find('ymin').text)
            ymax = float(member.find('bndbox').find('ymax').text)
            
            x_min = (xmin/wt)*self.width
            x_max = (xmax/wt)*self.width
            y_min = (ymin/ht)*self.height
            y_max = (ymax/ht)*self.height
            
            boxes.append([x_min, y_min, x_max, y_max])
            
        # Conversion to Tensors   
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])  # Calculating area of the boxes
        
        iscrowd = torch.zeros((boxes.shape[0], ), dtype=torch.int64)
        
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        image_id = torch.tensor([idx])
        
        target = {'boxes': boxes, 'area': area, 'labels': labels, 
                'iscrowd': iscrowd, 'image_id':image_id}
        
        if self.transforms:
            sample = self.transforms(image = img,
                                    bboxes = target['boxes'],
                                    labels = labels)
            
            img = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
            
        return img, target

In [ ]:
# Observing the dataset without any transformation
dataset = VehiclesDetectionDataset(files_dir, 224, 224)
print('length of dataset = ', len(dataset), '\n')

In [ ]:
img, target = dataset[78]
print(img.shape, '\n',target)

In [ ]:
def plot_img_bbox(img, target):
    
    # plot the image and bboxes
    # Bounding boxes are defined as follows: x-min y-min width height
    fig, a = plt.subplots(1,1)
    fig.set_size_inches(5,5)
    a.imshow(img)
    
    label_count = 0
    for box in (target['boxes']):
        x, y, width, height  = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = patches.Rectangle((x, y),
                                 width, height,
                                 linewidth = 2,
                                 edgecolor = 'r',
                                 facecolor = 'none')
        if(target['labels'][label_count]) == 1:
            write = 'car'
        if(target['labels'][label_count]) == 2:
            write = 'motorcycle'
        if(target['labels'][label_count]) == 3:
            write = 'bus'
        if(target['labels'][label_count]) == 4:
            write = 'truck'
        label_count = label_count+1
        
        a.annotate(write, (box[0]+5, box[1]-5), color='blue', weight='bold', fontsize=10, ha='center', va='center')
        # Draw the bounding box on top of the image
        a.add_patch(rect)
    plt.show()

In [ ]:
img, target = dataset[25]
plot_img_bbox(img, target)

In [ ]:
img, target = dataset[78]
plot_img_bbox(img, target)

In [ ]:
img, target = dataset[120]
plot_img_bbox(img, target)

In [ ]:
def get_model(num_classes, modelName):
    
    # Loading the pre-trained model
    if modelName == 'fastcnn':
        model = fasterrcnn_resnet50_fpn(pretrained=True)
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        return model
    
    elif modelName == 'maskcnn':
        model = maskrcnn_resnet50_fpn(pretrained=True)
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        return model

In [ ]:
def get_transform(train=True):
    if train:
        return A.Compose([
            A.HorizontalFlip(0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.RandomFog(p=0.5, fog_coef_lower=0.2, fog_coef_upper=0.4),
            A.GaussianBlur(p=0.5),
            ToTensorV2(p=0.1),     # ToTensorV2 converts image to PyTorch tensor without dividing by 255
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
    else:
        return A.Compose([
            ToTensorV2(p=0.1),     # ToTensorV2 converts image to PyTorch tensor without dividing by 255
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

In [ ]:
test_split = 0.2

# Loading the training and the testing data with all tghe transformations
dataset_train = VehiclesDetectionDataset(files_dir, 480, 480, transforms=get_transform(train=True))
dataset_test = VehiclesDetectionDataset(files_dir, 480, 480, transforms=get_transform(train=False))

torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()

# Train test split
tsize = int(len(dataset) * test_split) # Getting the splitting index
dataset_train = torch.utils.data.Subset(dataset_train, indices[:-tsize])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-tsize:])

# Defining dataloaders
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=8, shuffle=True,
                                              num_workers=4, collate_fn=utils.collate_fn)  # Imported form helper library
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=8, shuffle=True,
                                              num_workers=4, collate_fn=utils.collate_fn)

---

# 5. **Proposed solution** 

This is where you should implement most of the code for your solution. Write the routines for training and predicting the models and any necessary intermediate steps. Post-processing functions must also be implemented here.

  - Use good programming practices, modularizing and adequately commenting on your code. Code quality will be considered in the final assessment.
Again, we recommend using [PyTorch](https://pytorch.org/tutorials/beginner/introyt.html), but you are free to use any library or platform. You just need to justify that in the [final report](#scrollTo=ws14iV4Dp_vf).

  - You can use pre-trained models as backbones or any code available on the web as a basis, but they must be correctly credited and referenced both in this notebook and in the final report. Cite the source link repository and explicitly cite the authors of it.
If you changed existing code, make it clear what the changes were.
Make it clear where your own code starts and where it ends. Note that the originality percentage of the code will be considered in the evaluation, so use external codes wisely and sparingly. **Missconduct alert:** remember that there are many tools that compare existing source code and that it is relatively easy to identify authorship. So, be careful and fair by always properly thanking the authors if you use external code.

[top](#scrollTo=4i5afvUbhmGo)

In [ ]:
# Write your proposed solution code here. Create more code cells if you find it necessary
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 5 # Can try by changing to 3 as 1 class is devoted for background
num_epochs = 25

In [ ]:
def start_training(modelName, num_epochs, num_classes):
    loss = []
    loss_classifier = []
    loss_box_reg = []
    loss_objectness = []

    val_map = []
    val_map_05 = []

    model = get_model(num_classes, modelName)
    model.to(device)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.005)

    # Learning rate decreases by 10 every 5 epochs
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    
    for epoch in range(num_epochs):
        metric = train_one_epoch(model, optimizer, dataloader_train, device, epoch, print_freq=5)
        loss.append(metric.meters['loss'].value)
        loss_classifier.append(metric.meters['loss_classifier'].value)
        loss_box_reg.append(metric.meters['loss_box_reg'].value)
        loss_objectness.append(metric.meters['loss_objectness'].value)

        lr_scheduler.step()
        eval, stats = evaluate(model, dataloader_test, device=device)
        val_map.append(stats[0])
        val_map_05.append(stats[1])
    return model, loss, loss_classifier, loss_box_reg, loss_objectness, val_map, val_map_05

---

# 6. **Experimental tests and evaluations** 


Here you must implement your code for training, testing and evaluating your solution. For this, the following code blocks (*E1*, *E2*, and *E3*) are mandatory:

  - *E1* - Training the models. Implement code to call the dataloaders implemented for training your models.  Make routines to test different parameters of your models. Plot graphs that illustrate how parameters impact model training. Compare. Train and select a model for each city (A and B) and justify. You should use half (50%) of the samples from each dataset for training and leave the other half for testing (50%). 

[top](#scrollTo=4i5afvUbhmGo)

In [ ]:
# Write your codes for E1 here. Create more code cells if needed
fast_rcnn, loss, loss_classifier, loss_box_reg, loss_objectness, val_map, val_map_05 = start_training('fastcnn', num_epochs, num_classes)

In [ ]:
x_epochs = range(1, len(val_map) + 1)
plt.plot(x_epochs, val_map, label = "mAP@0.50:0.95")
plt.plot(x_epochs, val_map_05, label = "mAP@0.50")
plt.title("Validation mAP")
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('mAP')
plt.show()

In [ ]:
x_epochs = range(1, len(loss_classifier) + 1)
plt.plot(x_epochs, loss_classifier, label = "loss_classifier")
plt.plot(x_epochs, loss_box_reg, label = "loss_box_reg")
plt.plot(x_epochs, loss_objectness, label = "loss_objectness")
plt.title("Training Losses")
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
x_epochs = range(1, len(loss) + 1)
plt.plot(x_epochs, loss, label = "loss")
plt.title("Training Average Losses")
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Average Loss')
plt.show()

In [ ]:
def apply_nms(prediction, threshold):
    # torchvision returns the indices of the boxes to keep
    keep = torchvision.ops.nms(prediction['boxes'], prediction['scores'], threshold)
    
    final_prediction = prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]
    
    return final_prediction

In [ ]:
# Function to convert a torch tensor to a PIL Image
def tensorToPIL(img):
    return transforms.ToPILImage()(img).convert('RGB')

In [ ]:
# pick one image from the test set
img, target = dataset_test[5]

# put the model in evaluation mode
fast_rcnn.eval()
with torch.no_grad():
    prediction = fast_rcnn([img.to(device)])[0]
    
print('predicted #boxes: ', len(prediction['labels']))
print('real #boxes: ', len(target['labels']))

In [ ]:
print('EXPECTED OUTPUT')
plot_img_bbox(tensorToPIL(img), target)

In [ ]:
prediction['boxes'] = prediction['boxes'].cpu()
prediction['labels'] = prediction['labels'].cpu()
prediction['scores'] = prediction['scores'].cpu()
print('MODEL OUTPUT')
plot_img_bbox(tensorToPIL(img), prediction)

In [ ]:
nms_preds = apply_nms(prediction, threshold=0.2)
print('NMS APPLIED MODEL OUTPUT')
plot_img_bbox(tensorToPIL(img), nms_preds)

In [ ]:
test_dataset = VehiclesDetectionDataset(test_dir, 480, 480, transforms= get_transform(train=True))
# pick one image from the test set
img, target = test_dataset[10]
# put the model in evaluation mode
fast_rcnn.eval()
with torch.no_grad():
    prediction = fast_rcnn([img.to(device)])[0]
    prediction['boxes'] = prediction['boxes'].cpu()
    prediction['labels'] = prediction['labels'].cpu()
    prediction['scores'] = prediction['scores'].cpu()
print('EXPECTED OUTPUT\n')
plot_img_bbox(tensorToPIL(img), target)
print('MODEL OUTPUT\n')
nms_prediction = apply_nms(prediction, threshold=0.01)

plot_img_bbox(tensorToPIL(img), nms_prediction)

In [ ]:
!pip install torchmetrics

In [ ]:
import torch
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from pprint import pprint

In [ ]:
pred_list = []
target_list = []
for img, target in test_dataset:
  target_list.append(target)
  with torch.no_grad():
      prediction = fast_rcnn([img.to(device)])[0]
      prediction['boxes'] = prediction['boxes'].cpu()
      prediction['labels'] = prediction['labels'].cpu()
      prediction['scores'] = prediction['scores'].cpu()
      pred_list.append(prediction)

In [ ]:
metric = MeanAveragePrecision(class_metrics=True)
metric.update(pred_list, target_list)
pprint(metric.compute())

  - *E2* - Testing the models in the dataset. You must implement code routines to test the predictive ability of your models using half of each dataset intended for testing. **The model trained in city A must be tested in city A. The model trained in city B must be tested in city B.** Use the evaluation metrics (accuracy, F1-score, AUC, etc) that are most appropriate for your problem. Plot graphs that illustrate the results obtained for each city (A and B). Plot visual examples of correctly (true positive) and incorrectly (false positive) classified samples. 

[top](#scrollTo=4i5afvUbhmGo)


In [ ]:
# Write your codes for E2 here. Create more code cells if needed



  - *E3* - Testing the models crossing datasets. Here you must do exactly the same as in *E2*, but now training in one city and testing in the other. **The model trained in city A must be tested in city B. The model trained in city B must be tested in city A.** Use the same metrics and plot the same types of graphs so that results are comparable.

[top](scrollTo=4i5afvUbhmGo)

In [ ]:
# Write your codes for E3 here. Create more code cells if needed





---

# 7. **Quiz and Report**

Answer the assessment quiz that will be made available on Canvas one week before the final deadline. Make a 2-page latex report using the [IEEE template](https://www.overleaf.com/read/hkfsjjsxmxcn) with a maximum of 1000 words. You can deliver the report in MS Word if you prefer. Your report should contain five sections: introduction, description of the proposed solution with justifications, results (here you can include the same graphs and pictures generated in this jupyter notebook), discussion of the results, and conclusion. Properly cite references to articles, tutorials, and sources used. A pdf version of your report should be made available in the project's github repository under the name "[project name] + _final_report.pdf".


[top](#scrollTo=4i5afvUbhmGo)


---

# 8. **Demonstration**

Some projects (around 10%) will be selected for a mandatory demonstration. During the demo, you will be asked about implementation details and decisions that led to the design of the developed solution.

[top](#scrollTo=4i5afvUbhmGo)